In [ ]:
import os
import pandas as pd
import datetime as dt
import time
import calendar
import numpy as np
from statistics import mean


from uce_resources import get_site_id, get_mms_data, get_applied_forecast, get_prices, get_green_tariff


In [ ]:
from settings.sites import ceg_ as sites_list

target_year = 2023
target_month = 8
forecasts_types = ['real', 'naive', 'zero', 'increased_10_forecast_data', 'increased_20_forecast_data', 'increased_30_forecast_data', 
                   'increased_40_forecast_data', 'decreased_10_forecast_data']

target_folder = 'data/results/{}-{:0>2}/'.format(target_year, target_month)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# sites_list = ['Bar', 'Balky']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

# Data preparation section

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
with engine.connect() as connection:
    prices = get_prices(target_year, target_month, connection, metadata.tables['electricity_market_prices'], currency='UAH')

# price_dir = 'data/results/2022-10/'
# price_file = 'prices_2022_10_1-25.xlsx'
# prices = pd.read_excel(price_dir + price_file, index_col= 0)

min_price_day = (prices.index.min() + dt.timedelta(days=1)).day
max_price_day = prices.index.max().day

start_date = dt.date(year=target_year, month=target_month, day=min_price_day)
end_date = dt.date(year=target_year, month=target_month, day=max_price_day)
print(start_date, end_date)

prices.to_excel(target_folder + 'prices_{}_{}_{}-{}.xlsx'.format(target_year, target_month, min_price_day, max_price_day))
prices

In [ ]:

with engine.connect() as connection:
        
    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)

        site_data['green_tariff'] = get_green_tariff(site_data['site_id'], dt.date(year=target_year, month=target_month, day=1),
                                                     connection, metadata.tables['green_tariffs'], currency='UAH')
        print('Green tariff: {}'.format(site_data['green_tariff']))
        
        first_date = dt.date(target_year, target_month, 1)
        last_date = dt.date(target_year, target_month, calendar.monthrange(target_year, target_month)[1])

        mms_data, site_data['mms_version'] = get_mms_data(site_data['site_id'], 
                                                          target_year, target_month, 
                                                          connection, metadata.tables['mms_data'], include_prev=True,)
        # print(mms_data)
        print('MMS data | {} version | of | {} records |'.format(site_data['mms_version'], len(mms_data)))
        
        applied_forecast = get_applied_forecast(site_data['site_id'], start_date, end_date, 
                                                connection=connection, db_table=metadata.tables['forecasting_data'])
        print('Forecast data of | {} records |'.format(len(applied_forecast)))

        site_data['real_forecast_data'] = pd.concat([mms_data, applied_forecast], axis=1, join='inner')
        print('Real forecast data prepared')


        raw_forecast = get_applied_forecast(site_data['site_id'], start_date, end_date, 
                                            connection=connection, db_table=metadata.tables['forecasting_data'],
                                            forecast_type='forecast_applied_raw')
        print('Raw forecast data of | {} records |'.format(len(raw_forecast)))
 
        
        site_data['raw_forecast_data'] = pd.concat([mms_data, raw_forecast], axis=1, join='inner')
        print('Raw forecast data prepared')



        increased_10_forecast_data = raw_forecast * 1.1
        print('increased_10 forecast data of | {} records |'.format(len(increased_10_forecast_data)))
        
        site_data['increased_10_forecast_data'] = pd.concat([mms_data, increased_10_forecast_data], axis=1, join='inner')
        print('increased_10 forecast data prepared')


        increased_20_forecast_data = raw_forecast * 1.2
        print('increased_20 forecast data of | {} records |'.format(len(increased_20_forecast_data)))
        
        site_data['increased_20_forecast_data'] = pd.concat([mms_data, increased_20_forecast_data], axis=1, join='inner')
        print('increased_20 forecast data prepared')


        increased_30_forecast_data = raw_forecast * 1.3
        print('increased_30 forecast data of | {} records |'.format(len(increased_30_forecast_data)))
        
        site_data['increased_30_forecast_data'] = pd.concat([mms_data, increased_30_forecast_data], axis=1, join='inner')
        print('increased_30 forecast data prepared')


        increased_40_forecast_data = raw_forecast * 1.4
        print('increased_40 forecast data of | {} records |'.format(len(increased_40_forecast_data)))

        # raw_forecast.index=pd.to_datetime(raw_forecast.index)
        # mask=(raw_forecast.index.time >= pd.to_datetime('07:30:00').time()) & (raw_forecast.index.time >= pd.to_datetime('11:30:00').time())
        # increased_40_forecast_data=raw_forecast.copy()
        # increased_40_forecast_data.loc[mask, 'forecast [kWh]'] *=1.2
        # print('increased_40 forecast data of | {} records |'.format(len(increased_40_forecast_data)))
        
        site_data['increased_40_forecast_data'] = pd.concat([mms_data, increased_40_forecast_data], axis=1, join='inner')
        print('increased_40 forecast data prepared')


        decreased_10_forecast_data = raw_forecast * 0.9
        print('decreased_10 forecast data of | {} records |'.format(len(decreased_10_forecast_data)))
        
        site_data['decreased_10_forecast_data'] = pd.concat([mms_data, decreased_10_forecast_data], axis=1, join='inner')
        print('decreased_10 forecast data prepared')


        site_data['zero_forecast_data'] = pd.concat([mms_data, applied_forecast * 0], axis=1, join='inner')
        print('Zero forecast data prepared')

        naive_forecast_data = pd.concat([mms_data, mms_data.shift(48)], axis=1, join='inner').dropna(axis=0, how='any')
        naive_forecast_data.columns = ['yield [kWh]', 'forecast [kWh]']
        naive_forecast_data['forecast [kWh]'] = naive_forecast_data['forecast [kWh]'].astype(int)
        site_data['naive_forecast_data'] = naive_forecast_data
        print('Naive forecast data prepared')
        
        sites_data.update({site: site_data})
        end = time.time()

        print('Processing took {} seconds'.format(round(end - start, 2)))


## Unbalance cost estimations

In [ ]:
columns = ['site', 'first_date', 'last_date', 'hour', 'region', 'cluster',
            'forecast_type','revenue [UAH]', 
            'error_u [kWh]', 'error_u [%]',
            'error_u (excess) [kWh]',
            'error_u (shortage) [kWh]',
            'cieq_641_rule (excess) [UAH]', 
            'cieq_641_rule (shortage) [UAH]', 
            'cieq_641_rule (net) [UAH]',
            'cieq_641_rule* [UAH]', 
            'cieq_641_rule_positive* [UAH]', 'cieq_641_rule_negative* [UAH]']

### Daily results

In [ ]:
daily_indexes = list()

for day in range(1, calendar.monthrange(target_year, target_month)[-1] + 1):
    start = dt.datetime(year=target_year, month=target_month, day=day, hour=0, minute=30)
    end = dt.datetime(year=target_year, month=target_month, day=day, hour=23, minute=30)
    index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
    index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)
    daily_indexes.append(index_in_utc)

print(len(daily_indexes))

In [ ]:
print(daily_indexes[0])

In [ ]:
import importlib
imported_module = importlib.import_module("uce_resources")
importlib.reload(imported_module)
from uce_resources import make_results_hourly

results_real = pd.DataFrame(columns=columns)
results_raw = pd.DataFrame(columns=columns)
results_naive = pd.DataFrame(columns=columns)
results_zero = pd.DataFrame(columns=columns)
results_increased_10 = pd.DataFrame(columns=columns)
results_increased_20 = pd.DataFrame(columns=columns)
results_increased_30 = pd.DataFrame(columns=columns)
results_increased_40 = pd.DataFrame(columns=columns)
results_decreased_10 = pd.DataFrame(columns=columns)

with engine.connect() as connection:
    for site in sites_data.keys():
        
        sites_table = metadata.tables['sites']
        list_to_select = [
            sites_table.c.id,
            sites_table.c.region, 
            sites_table.c.cluster
            ]
        query = select(list_to_select).where(sites_table.c.displayable_name == site)
        site_id_response = connection.execute(query).fetchall()[0]

        site_id = site_id_response[0]
        region = site_id_response[1]
        cluster = site_id_response[2]


        for index in daily_indexes:
            # print(sites_data[site]['real_forecast_data'])
            result_real = make_results_hourly(sites_data[site], 'real', prices, index, region, cluster)
            # print(result_real)
            if not result_real is None:
                results_real = results_real.append(result_real, ignore_index=True)

            result_raw = make_results_hourly(sites_data[site], 'raw', prices, index, region, cluster)
            # print(result_raw)
            if not result_raw is None:
                results_raw = results_raw.append(result_raw, ignore_index=True)    

            result_naive = make_results_hourly(sites_data[site], 'naive', prices, index, region, cluster)      
            #print(result_naive)
            if not result_naive is None:
                results_naive = results_naive.append(result_naive, ignore_index=True)

            result_zero = make_results_hourly(sites_data[site], 'zero', prices, index, region, cluster)      
            #print(result_zero)
            if not result_zero is None:
                results_zero = results_zero.append(result_zero, ignore_index=True)

            result_increased_10 = make_results_hourly(sites_data[site], 'increased_10', prices, index, region, cluster)      
            #print(result_increased_10)
            if not result_increased_10 is None:
                results_increased_10 = results_increased_10.append(result_increased_10, ignore_index=True)

            result_increased_20 = make_results_hourly(sites_data[site], 'increased_20', prices, index, region, cluster)      
            #print(result_increased_20)
            if not result_increased_20 is None:
                results_increased_20 = results_increased_20.append(result_increased_20, ignore_index=True)

            result_increased_30 = make_results_hourly(sites_data[site], 'increased_30', prices, index, region, cluster)      
            #print(result_increased_30)
            if not result_increased_30 is None:
                results_increased_30 = results_increased_30.append(result_increased_30, ignore_index=True)

            result_increased_40 = make_results_hourly(sites_data[site], 'increased_40', prices, index, region, cluster)      
            #print(result_increased_40)
            if not result_increased_40 is None:
                results_increased_40 = results_increased_40.append(result_increased_40, ignore_index=True)

            result_decreased_10 = make_results_hourly(sites_data[site], 'decreased_10', prices, index, region, cluster)      
            #print(result_decreased_10)
            if not result_decreased_10 is None:
                results_decreased_10 = results_decreased_10.append(result_decreased_10, ignore_index=True)



        sites_data[site]['results_real'] = results_real
        sites_data[site]['results_raw'] = results_raw
        sites_data[site]['results_naive'] = results_naive
        sites_data[site]['results_zero'] = results_zero
        sites_data[site]['results_increased_10'] = results_increased_10
        sites_data[site]['results_increased_20'] = results_increased_20
        sites_data[site]['results_increased_30'] = results_increased_30
        sites_data[site]['results_increased_40'] = results_increased_40
        sites_data[site]['results_decreased_10'] = results_decreased_10


        print(f'{site} - Results daily: Ok!')

    # print(sites_data[site]['results_real'])

In [ ]:
from uce_resources import save_results, format_excel

results_daily = pd.concat([results_real, results_raw, results_naive, results_zero, results_increased_10, results_increased_20, 
                           results_increased_30, results_increased_40, results_decreased_10], ignore_index=True, axis=0)

# min_day = results_daily.first_date.min().day
# max_day = results_daily.last_date.max().day

with pd.ExcelWriter(target_folder + 'uce_daily_hourly_{}_{}_UAH.xlsx'.format(target_year, target_month), engine="openpyxl") as  writer:
    results_daily.to_excel(writer, 'results_daily')

#  writer.save()
# format_excel(writer, results_daily).save()

print('Saving results: ok!')
